### 1. 가상의 브라우저 실행하기: Chrome Driver

---

In [ ]:
# Python 코드를 통해 가상의 브라우저를 띄우기 위해 selenium 패키지를 import 합니다.
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# selenium을 활용해 브라우저를 직접 띄우는 경우, 실제 웹서핑을 할때처럼 로딩시간이 필요합니다.
# 로딩시간 동안 대기하도록 코드를 구성하기위해 time 패키지를 import 합니다.
import time

# Python 코드를 통해 웹페이지에 정보를 요청하기 위해 BeautifulSoup, urllib 패키지를 import 합니다.
from bs4 import BeautifulSoup
import requests

In [ ]:
# Chrome Driver를 호출합니다.
chrome_options = webdriver.ChromeOptions()

# 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨깁니다.
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')

# Chrome Driver 파일의 경로를 지정하고 실행합니다.
# Chrome Driver는 아래 링크에서 다운로드 가능합니다.
# 본 Jupyter Notebook 파일과 동일한 경로에 Chrome Driver가 존재하는 경우 아래 경로를 그대로 사용합니다.

#service = Service("chromedriver.exe")    # Windows 운영체제
service = Service("./chromedriver")    # MAC, Linux 운영체제
                                       # 경고메시지 출력시 조치 : [시스템 환경설정] > [보안 및 개인정보 보호] > "Chrome Drive ~ 확인없이 허용"

driver = webdriver.Chrome(service=service, options=chrome_options)

### 2. 리뷰 페이지 이동하기

In [ ]:
URL = "https://play.google.com/store/apps/details?id=com.coupang.mobile.play&showAllReviews=true"
driver.get(URL)

In [ ]:
driver.find_elements(By.CLASS_NAME, "VfPpkd-vQzf8d")[-1].click()    
# "설치", "리뷰 정책 및 정보", "리뷰 모두 보기" 모두 동일한 Class 속성을 가짐
# .find_elements()로 세 버튼을 모두 가져와 마지막 버튼 클릭

### 3. 리뷰 스크롤 내리기

In [ ]:
# 스크롤이 동작하지 않는 경우에만 선택적으로 적용합니다.
# 크롬창을 최대화합니다.
#driver.maximize_window()

scroll_count = 10

review_div = driver.find_element(By.CLASS_NAME, "fysCi")
for i in range(scroll_count):
    driver.execute_script("arguments[0].scrollTo(0, document.body.scrollHeight+999999999999)", review_div)
    time.sleep(1)

In [ ]:
# 스크롤을 다시 맨 위로 이동
driver.execute_script('arguments[0].scrollTo(0, 0);', review_div)

review_box_list = driver.find_elements(By.CLASS_NAME, "RHo1pe")
print("> 리뷰개수 :", len(review_box_list))

### 4. 리뷰 수집하기

In [ ]:
f = open("coupang_play.txt", "w", encoding = "utf-8")

progress = 0 
for review_box in review_box_list:
    progress += 1
    print(progress, end="\r")
    
    # 별점
    review_star = review_box.find_element(By.CLASS_NAME, "iXRFPc")        
    star_label = review_star.get_attribute("aria-label")
    # 작성일자
    review_date = review_box.find_element(By.CLASS_NAME, "bp9Aid").text
    # 리뷰 텍스트
    review_text = review_box.find_element(By.CLASS_NAME, "h3YV2d").text
    
    # 파일에 기록하기 + 출력하기
    f.write(review_date + "\t" + star_label + "\t" + review_text + "\n")
    #print(review_date + "\t" + star_label + "\t" + review_text)
    
f.close()

In [ ]:
# 파일에 저장된 카페 게시글 내용을 확인합니다.
f = open("coupang_play.txt", encoding="utf-8")
for review in f:
    print(review.strip())
f.close()

In [ ]:
driver.close()